## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql

SELECT name
FROM facilities
WHERE membercost > 0


name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(*)
FROM facilities
WHERE membercost == 0

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql

SELECT facid, name, membercost, monthlymaintenance
FROM facilities
WHERE membercost / monthlymaintenance < 0.2


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql

SELECT *
FROM facilities
WHERE facid == 1

UNION

SELECT *
FROM facilities
WHERE facid == 5

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql

SELECT name, monthlymaintenance, 'expensive' AS description
FROM facilities
WHERE monthlymaintenance > 100

UNION

SELECT name, monthlymaintenance, 'cheap' AS description
FROM facilities
WHERE monthlymaintenance <= 100

name,monthlymaintenance,description
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Pool Table,15,cheap
Snooker Table,15,cheap
Badminton Court,50,cheap
Squash Court,80,cheap
Table Tennis,10,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql

SELECT firstname, surname
FROM members
WHERE joindate == (SELECT max(joindate) FROM members)

firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql

SELECT concat(M.surname, ', ', M.firstname) AS name, Q.name AS facility, Q.starttime
FROM members M, (
  SELECT *
  FROM facilities F, bookings B
  WHERE F.name LIKE 'Tennis Court%'
  AND F.facid == B.facid
) Q
WHERE M.memid == Q.memid
ORDER BY concat(M.surname, ', ', M.firstname) ASC, Q.starttime DESC

name,facility,starttime
"Bader, Florence",Tennis Court 2,2012-09-23 15:00:00
"Bader, Florence",Tennis Court 1,2012-09-23 11:00:00
"Bader, Florence",Tennis Court 2,2012-09-22 14:00:00
"Bader, Florence",Tennis Court 2,2012-09-10 14:30:00
"Bader, Florence",Tennis Court 2,2012-09-09 16:00:00
"Bader, Florence",Tennis Court 2,2012-09-02 08:30:00
"Bader, Florence",Tennis Court 2,2012-08-25 18:30:00
"Bader, Florence",Tennis Court 2,2012-08-23 19:00:00
"Bader, Florence",Tennis Court 2,2012-08-18 12:30:00
"Baker, Anne",Tennis Court 2,2012-09-29 14:30:00


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql

SELECT
  concat(M.surname, ', ', M.firstname) AS name,
  F.name AS facility,
  F.membercost * CAST(B.memid != 0 AS INT) + F.guestcost * CAST(B.memid == 0 AS INT) AS cost
FROM facilities F, bookings B, members M
WHERE year(B.starttime) == 2012
AND month(B.starttime) == 9
AND day(B.starttime) == 14
AND F.facid == B.facid
AND B.memid == M.memid
AND (F.membercost * CAST(B.memid != 0 AS INT) + F.guestcost * CAST(B.memid == 0 AS INT)) > 30
ORDER BY cost DESC


name,facility,cost
"GUEST, GUEST",Massage Room 1,80.0
"GUEST, GUEST",Massage Room 1,80.0
"GUEST, GUEST",Massage Room 1,80.0
"GUEST, GUEST",Massage Room 2,80.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql

SELECT name, facility, cost
FROM (
  (
    SELECT
      B.bookid,
      concat(M.surname, ', ', M.firstname) AS name,
      F.name AS facility,
      F.membercost AS cost
    FROM facilities F, bookings B, members M
    WHERE year(B.starttime) == 2012
    AND month(B.starttime) == 9
    AND day(B.starttime) == 14
    AND F.facid == B.facid
    AND B.memid == M.memid
    AND F.membercost > 30
    AND B.memid != 0
  ) UNION (
    SELECT
      B.bookid,
      concat(M.surname, ', ', M.firstname) AS name,
      F.name AS facility,
      F.guestcost AS cost
    FROM facilities F, bookings B, members M
    WHERE year(B.starttime) == 2012
    AND month(B.starttime) == 9
    AND day(B.starttime) == 14
    AND F.facid == B.facid
    AND B.memid == M.memid
    AND F.guestcost > 30
    AND B.memid == 0
  )
)
ORDER BY cost DESC

name,facility,cost
"GUEST, GUEST",Massage Room 1,80.0
"GUEST, GUEST",Massage Room 1,80.0
"GUEST, GUEST",Massage Room 2,80.0
"GUEST, GUEST",Massage Room 1,80.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql

SELECT
  FIRST(F.name) AS facility,
  SUM(F.membercost * CAST(B.memid != 0 AS INT) + F.guestcost * CAST(B.memid == 0 AS INT)) AS totalrevenue
FROM facilities F, bookings B
WHERE F.facid == B.facid
GROUP BY F.facid
ORDER BY totalrevenue DESC

facility,totalrevenue
Massage Room 1,20807.900000000045
Massage Room 2,6987.299999999997
Squash Court,4970.0
Tennis Court 2,4205.0
Tennis Court 1,4040.0
Badminton Court,604.5
Pool Table,265.0
Snooker Table,115.0
Table Tennis,90.0
